In [6]:
import tkinter as tk
from tkinter import filedialog

class SelectFoldersDialog:
    def __init__(self, master):
        self.master = master
        self.selected_folders = []

        self.label = tk.Label(master, text="Sélectionnez les dossiers :")
        self.label.pack()

        self.add_button = tk.Button(master, text="Ajouter un dossier", command=self.add_folder)
        self.add_button.pack()

        self.confirm_button = tk.Button(master, text="Confirmer la sélection", command=self.confirm_selection)
        self.confirm_button.pack()

    def add_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.selected_folders.append(folder)
            print("Dossier ajouté :", folder)

    def confirm_selection(self):
        if self.selected_folders:
            print("Sélection confirmée :", self.selected_folders)
            self.master.destroy()
        else:
            print("Aucun dossier sélectionné. Veuillez ajouter au moins un dossier.")

            # Si aucun dossier n'est sélectionné, vous pouvez choisir de quitter l'application ici
            # self.master.destroy()

def open_select_folders_dialog():
    root = tk.Tk()
    dialog = SelectFoldersDialog(root)
    root.mainloop()

    return dialog.selected_folders  # Retourner les dossiers sélectionnés

# Exemple d'utilisation : ouvrir une fenêtre de dialogue pour sélectionner les dossiers
selected_folders = open_select_folders_dialog()

# Utiliser les dossiers sélectionnés ailleurs dans votre programme
print("Dossiers sélectionnés :", selected_folders)

Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a53d1s1_ephys/21a53d1s1_210604_150840
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a51d1s1_ephys/21a51d1s1_210602_110516
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a21d1s1_ephys/21a21d1s1_210408_155731
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a19d1s1_ephys/21a19d1s1_210403_130057
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a18d1s1_ephys/21a18d1s1_210401_111241
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a17d1s1_ephys/21a17d1s1_210330_130118
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a15d1s1_ephys/21a15d1s1_210326_163524
Sélection confirmée : ['P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a53d1s1_ephys/21a53d1s1_210604_150840', 'P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a51d1s1_ep

In [13]:
import os
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert import NotebookExporter
import nbformat
import jdata as jd
from tqdm import tqdm  # Importer tqdm pour la barre de progression

# Nombre total de dossiers à analyser
total_folders = len(selected_folders)

# Utiliser tqdm pour afficher une barre de progression
for i, selected_path in enumerate(tqdm(selected_folders, desc="Analyzing folders", unit="folder")):
    executed_notebook = os.path.join(r'C:\Users\gbouvier\BOUVIER', 'Analyze.ipynb')
    result_notebook = os.path.join(r'C:\Users\ayazici\Documents\Analyses', f"Analyze_{i+1}.ipynb")  

    jd.save(selected_path, os.path.join(r'C:\Users\ayazici\Documents\Analyses', 'path.json'))

    # Charger le notebook à exécuter
    with open(executed_notebook, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    # Créer un préprocesseur pour exécuter le notebook
    preprocessor = ExecutePreprocessor(timeout=None)

    # Exécuter le notebook
    preprocessor.preprocess(notebook)

    # Exporter le notebook exécuté
    exporter = NotebookExporter()
    body, resources = exporter.from_notebook_node(notebook)

    # Écrire le notebook exécuté dans un fichier
    with open(result_notebook, 'w', encoding='utf-8') as f:
        f.write(body)

    print(f"Analysis done")



Analyzing folders:  14%|█▍        | 1/7 [07:50<47:04, 470.67s/folder]

Analysis done
